# Compute cost distance surfaces
> This lesson serves as an example showing the utility of the NumPy package in spatial analysis. Here, we use Numpy mostly to handle our data so that another powerful package -- SciKit-Image -- can access it and do some speedy cost distance analysis. However, this lesson does illustrate how both vectors and rasters can be converted to numpy arrays, the versaility of the numpy package for wrangling and visualizing multidimensional data, and how these numpy arrays can be converted back to familiar formats. 

This script iterates through each point feature in a point feature layer and computes the cost distance away from that point using the supplied cost surface raster. Each cost surface is then added to a stack of cost distance and cost path rasters, in the form of a NumPy raster stack. With this stack we can extract single cost distance surfaces or reduce the stack to a single stack, e.g. take the minimum value across all layers, and visualize that. 

In [ ]:
#Imports
import arcpy
import numpy as np      

#For minimum cost path analysis
from skimage import graph  

#For 3D plotting
from matplotlib import pyplot as plt
from matplotlib import cm                   # colormaps for figures
from mpl_toolkits.mplot3d import Axes3D     # to construct 3d figures

#### 1. Convert the cost surface to a numpy array
* https://pro.arcgis.com/en/pro-app/arcpy/functions/rastertonumpyarray-function.htm
* https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.imshow.html

In [ ]:
#Read the MIT cost surface into a numpy array
MIT_cost_surface = '../data/MIT_Surface_Subset/MIT_CostSurface_500m.img'
arrCost = arcpy.RasterToNumPyArray(
    in_raster=MIT_cost_surface,
    nodata_to_value=-1)
arrCost.shape

In [ ]:
#Show the numpy array
plt.imshow(arrCost);

#### 2. Convert point feature class to a numpy _structured_ array
* https://pro.arcgis.com/en/pro-app/arcpy/data-access/featureclasstonumpyarray.htm

In [ ]:
#Convert the features to a numpy structured array
arrBiogas = arcpy.da.FeatureClassToNumPyArray(
    in_table='../data/DuplinSites_Prj.shp',
    field_names=['FID','SHAPE@X','SHAPE@Y','Cost_USDmi']
)

In [ ]:
#Show the first two rows of data
arrBiogas[:2]

In [ ]:
#Show the first 4 rows a particular column of data
arrBiogas['Cost_USDmi'][:4]

#### 3. Translate projected biogas locations into image coordinates
Since numpy and the scikit-image processes don't do geospaital projections, we have to translate our earth-bound coordinates to image coordinates, i.e., those referenced to the upper-left (Xmin,Ymax) corner of the cost surface. We use the affine transormation to do this, ignoring any distortion...

In [ ]:
#Convert projected coordinates to image coordinates using affine transformation
## Get the description of the cost surface raster
imgDescribe = arcpy.da.Describe(MIT_cost_surface)

## Get the extent and its compontents
imgExtent = imgDescribe['extent']
xMin = imgExtent.XMin
yMin = imgExtent.YMin
xMax = imgExtent.XMax
yMax = imgExtent.YMax

## Get the cell size of the image
cellX = imgDescribe['meanCellHeight']
cellY = imgDescribe['meanCellWidth']

## Compute X- and Y-offsets for each point, truncating to the nearest integer
xOffsets = np.trunc((arrBiogas['SHAPE@X'] - xMin)/cellX)
yOffsets = np.trunc((arrBiogas['SHAPE@Y'] - yMax)/cellY)

In [ ]:
#Stack the X and Y offsets; check values
arrXY = np.stack((xOffsets,yOffsets),axis=1)
arrXY[0],xOffsets[0],yOffsets[0]

#### 4a. Proof of concept: Compute cost distance and cost back link for one point
Before we do _all_ the points, let's do one and see how it goes. 

In [ ]:
#Compute the cost distance and path for the first point
##Get the coordinates (as offsets)
x,y = arrXY[0]

##Compute the minimum cost graph from the point
mcp_graph = graph.MCP_Geometric(arrCost)

##Compute the cost distances awayt from the source coordinates
arrCostDist, arrBackLink = mcp_graph.find_costs(starts=[(x,y)])

This output should be the same size as our cost raster...

In [ ]:
#Reveal the size of the cost distance array output
arrCostDist.shape

Show the arrays as images using the matplotlib `imshow` command.

In [ ]:
#Show the resulting costs surface
plt.imshow(arrCostDist) #Draw the data
plt.colorbar();         #Add a color bar legend

In [ ]:
#Show the resulting cost backlink
plt.imshow(arrBackLink,cmap='Set1');

A quick demo showing that we can plot these things in 3d...

In [ ]:

#Create a meshgrid that is the size of the cost distance array: this is the axis
y = np.arange(0,arrCostDist.shape[0])
x = np.arange(0,arrCostDist.shape[1])
X,Y  = np.meshgrid(x,y)

#Create the figure
fig = plt.figure(figsize=(20,5))            #Create the figure canvas
ax = fig.add_subplot(111, projection='3d')   #Add 3d plot...

#Plot contours
#ax.contour3D(X,Y,arrCostDist,100,alpha=0.2,cmap='ocean');

#Or, plot a surface
ax.plot_surface(X,Y,arrCostDist,
                linewidth=0,
                alpha=0.6);

#### 4b. Now that we did one point successfully, iterate through all 464 biogas sources
We'll collect each source's cost distance and backlink rasters into a list, and then convert these list to a stack of layers. 

In [ ]:
#Iteration through all points, compute the cost distance and backlink surfaces and add to a stack of surfaces
##Initialize the lists holding the two outputs
costDistance_surfaces = []
costBacklink_surfaces = []

##Iterate through each biogas source
for x,y in arrXY:
    #Show status
    print('.',end='')
    
    #Compute the minimum cost graph from the point
    mcp_graph = graph.MCP_Geometric(arrCost)

    #Compute the cost distances awayt from the source coordinates
    arrCostDist, arrBackLink = mcp_graph.find_costs(starts=[(x,y)])
    
    #Add to the list of arrays
    costDistance_surfaces.append(arrCostDist)
    costBacklink_surfaces.append(arrBackLink)

##Status
print("Completed!")
    
##Create stacked arrays
arrCostdistances = np.stack(costDistance_surfaces)
arrBacklinks = np.stack(costBacklink_surfaces)

We should now have a stack of 464 rasters, each 190 rows x 223 columns...

In [ ]:
#Display info about the stacks
arrCostdistances.shape

Now, we can extract a single layer from the stack and plot it (for example)...

In [ ]:
#Extract a single layer from the stack, we'll get #130
layer130 = arrCostdistances[130,:,:]

#Extract the point coordinates for this layer
theX,theY = arrXY[130]

#Plot the surface
ax = plt.imshow(layer130)
plt.colorbar();
#.scatter(theX,theY)

In [ ]:
#Now we'll plot that layer
fig = plt.figure(figsize=(20,5))            #Create the figure canvas
ax = fig.add_subplot(111, projection='3d')   #Add 3d plot...

#plot a surface
ax.plot_surface(X,Y,arrCostdistances[point_number,:,:],
                linewidth=0,
                alpha=0.6);

In [ ]:
#Compute the average cost distance for every pixel
minCostDistanceSurface = np.min(arrCostdistances,axis=0)

#Set all values above a cost threshold to NoData
minCostDistanceSurface[minCostDistanceSurface>4000]=np.NaN

In [ ]:
#Plot the mean, tilting and rotating the axis and setting the z range
fig = plt.figure(figsize=(20,15))            #Create the figure canvas
ax = fig.add_subplot(111, projection='3d')   #Add 3d plot...
ax.set_zlim(0, 8000)                        #Set the cost (z) range on the plot
ax.view_init(elev=30, azim=-20)              #Set the perspective

#Plot the surface
ax.plot_surface(X,Y,minCostDistanceSurface,
                linewidth=10,
                alpha=0.6);

#### 5. Saving the rasters
Here, we'll save the raster stack as a numpy export file and also convert the raster stack and the minimum raster back into a GIS ready (TIFF) file format. 
* Save a file to a binary NumPy object: https://docs.scipy.org/doc/numpy/reference/generated/numpy.save.html
* ArcPy's numpy to raster: https://pro.arcgis.com/en/pro-app/arcpy/functions/numpyarraytoraster-function.htm
 * BUG: Only exports 1st layer: https://community.esri.com/thread/237873-arcpy-numpyarraytoraster-not-working-properly-python
 * Alternative: use `gdal`...

In [ ]:
#Save the cost distance and backlink stacks to binary numpy files that can be re-imported later
np.save(file='../data/costdistances.npy',arr=arrCostdistances)
np.save(file='../data/costbacklinks.npy',arr=arrBacklinks)

* To write the surfaces to a GIS file format (e.g. GeoTIFF), we need to first create an empty tiff object and then fill it with our data. 

In [ ]:
#Convert the surface back to a raster, using values we extracted from the original cost surface
outRaster = arcpy.NumPyArrayToRaster(
    in_array=arrCostdistances,
    lower_left_corner = arcpy.Point(xMin,yMin),
    x_cell_size = cellX,
    y_cell_size = cellY
)

In [ ]:
#Save the raster
outRaster.save('../data/CostDistanceStack.tif')

If you open that raster in ArcPro, you'll see just one band created because of the bug. However, if we know GDAL, we can use that to do the same thing -- successfully!

In [ ]:
#Create the output TIF object and set its properties
import gdal

#Get projection info from the input raster
ds = gdal.Open(MIT_cost_surface)
ds_prj = ds.GetProjection()
ds_transform = ds.GetGeoTransform()

#Determine the size of the output TIFF
bands, height,width = arrCostdistances.shape
drv = gdal.GetDriverByName("GTiff")
dsOut = drv.Create('../data/cost_surface_stack.tif',width,height,bands,gdal.GDT_Float32)
dsOut.SetGeoTransform (ds_transform)  #Set the pixel size, offset, and warp
dsOut.SetProjection(ds_prj)           #Define the coordinate system

#Write to the data source object
for i in range(bands):
    dsOut.GetRasterBand(i+1).WriteArray(arrCostdistances[i,:,:]) #Write the data to the 1st band
    dsOut.GetRasterBand(i+1).SetNoDataValue(-9999.9)             #Assign NoData values
dsOut.FlushCache()